In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RBFInterpolator
from typing import Callable, Tuple, List
from scipy.linalg import cholesky, solve_triangular

In [2]:
path="/home/jorgebdelafuente/Doctorado/PH2M/Sings/"
dm_entry_11 = np.loadtxt(path+"entry/dm_1.1.dat")
dm_entry_12 = np.loadtxt(path+"entry/dm_1.2.dat")
dm_exit_11 = np.loadtxt(path+"exit/dm_1.1.dat")
dm_exit_12 = np.loadtxt(path+"exit/dm_1.2.dat")

In [3]:
def get_ic(rg,rp,theta):
    thrad=np.radians(theta)
    r12=((rp*0.50)*(rp*0.50)+rg*rg-2.0*0.50*rp*rg*np.cos(thrad))**0.50
    r13=((rp*0.50)*(rp*0.50)+rg*rg-2.0*0.50*rp*rg*np.cos(np.pi-thrad))**0.50
    r23=rp
    return r12,r13,r23

In [4]:
dm_11_entry_ic = np.zeros((len(dm_entry_11), 6))
dm_12_entry_ic = np.zeros((len(dm_entry_12), 6))

for i in range(len(dm_entry_11)):
    r12, r13, r23 = get_ic(dm_entry_11[i, 0], dm_entry_11[i, 1], dm_entry_11[i, 1])
    # coge las columnas 3,4,5 (índices 3:6)
    dm_11_entry_ic[i, :] = np.array([r12, r13, r23,
                                    dm_entry_11[i, 3], dm_entry_11[i, 4], dm_entry_11[i, 5]])

for i in range(len(dm_entry_12)):
    r12, r13, r23 = get_ic(dm_entry_12[i, 0], dm_entry_12[i, 1], dm_entry_12[i, 2])
    dm_12_entry_ic[i, :] = np.array([r12, r13, r23,
                                    dm_entry_12[i, 3], dm_entry_12[i, 4], dm_entry_12[i, 5]])


In [5]:
dm_mod_11 = np.zeros((len(dm_11_entry_ic)+len(dm_exit_11), 4))
dm_mod_12 = np.zeros((len(dm_12_entry_ic)+len(dm_exit_12), 4))

def fill_mod(out, first, second):
    n1 = len(first)
    out[:n1, :3] = first[:, :3]
    out[:n1,  3] = np.linalg.norm(first[:, 3:6], axis=1)
    out[n1:, :3] = second[:, :3]
    out[n1:,  3] = np.linalg.norm(second[:, 3:6], axis=1)

fill_mod(dm_mod_11, dm_11_entry_ic, dm_exit_11)
fill_mod(dm_mod_12, dm_12_entry_ic, dm_exit_12)

In [9]:
# Tus datos: distancias y momento dipolar
x = np.column_stack((dm_mod_11[:,0], dm_mod_11[:,1], dm_mod_11[:,2]))  # distancias
mu  = dm_mod_11[:,3]   # el módulo del dipolo

# Interpolador RBF
rbf_11_MQ = RBFInterpolator(x, mu, kernel="multiquadric", epsilon=1.0, degree=0, smoothing=1e-8)

# Prueba de método de análisis LOO y PRESS para RBF interpolation

##### Interpolación RBF

La interpolación RBF consiste en:

$ s(x) = \Sigma_j c_j \phi (\vert x -x_j \vert) $, 

dónde,  el "kernel" ($\phi$) se define como una función, que puede ser gaussiana, multicuadrática, cúbica, polinómica... En este problema me voy a centrar en:

- Multicuadrática: $\phi(r) = \sqrt(1+(\epsilon r)^2)$ 
- Inversa MC: $\phi(r) = 1/\sqrt(1+(\epsilon r)^2)$ 
- Gausiana: $\phi(r) = \exp(-(\epsilon r)^2)$
- Polinómicas de orden 3 y 5.

$\epsilon$ controla el alcance/curvatura de la función para cada punto.

El ajuste se hace resolviendo la matriz del Kernel ($K_{ij} = \phi(\vert x_i - x_j\vert;\epsilon)$) con:

$(K+\lambda I)c = y$. Si $M = (K+\lambda I)^{-1}$, entonces: $c=My$.

##### Problema 

Como es una interpolación, es muy díficil hacer un análisis correcto del ajuste, ya que los puntos originales del set, siempre van a dar un error de 0 o muy pequeño. Por lo tanto, una manera de analizar este problema es usando el método Leave-One-Out / Predicted Residual Sum of Squares (LOO/PRESS). 

El método LOO consiste en hacer excluir todos los puntos del set original 1 vez y comprobar con lo predicho en la interpolación, utilizando el estadístico $PRESS = \Sigma_i (e_i^{LOO})^2 = \Sigma_i (c_i/M_{ii})^2 $. El residulal LOO se define como $e_i^{LOO} = \hat{y}_i^{(-i)} - y_i = -c_i/M_{ii}$.

Sin embargo, no hace falta entrenar/realizar el ajuste N veces quitando un punto diferente cada vez, se puede hacer de manera eficiente haciendo uso de las relaciones que he mostrado anteriormente. Este método consiste en: 

- Factorizar la matriz del Kernel ($K_{\lambda} = K + \lambda I$) haciendo uso del método Cholesky, dónde: $K_\lambda = LL^T$ (L siendo la parte bajo la diagonal).
- Con las relaciones previas, se puede resolver $LL^Tc=y$ con dos sustituciones triangulares.
- La diagonal de M, siendo $M=L^{-T}L^{-1} \rightarrow M_{ii}=\vert \vert z_i \vert \vert ^2$, siendo $z_i$: $L z_i =e_i$. Siendo $e_i$ el vector canónico (unidad) con 1 en la i-ésima posición.
- Entonces, para cada i, se resuelve $L z_i =e_i$, computando $M_{ii} = z_i^Tz_i$

###### Por qué M_ii = z_i^T z_i?

- Cholesky: $K_λ = L L^T ⇒ M = K_λ^{-1} = L^{-T} L^{-1}$.
- Diagonal: $M_ii = e_i^T M e_i = e_i^T L^{-T} L^{-1} e_i = (L^{-1} e_i)^T (L^{-1} e_i)$.
- Define $z_i = L^{-1} e_i$, que se obtiene resolviendo $L z_i = e_i$ (sustitución hacia delante).
- Entonces $M_{ii} = ||z_i||^2 = z_i^T z_i$.

In [16]:
# Definición de los kernels

def kernel_factory(name:str,epsilon:float=1.0) -> Callable[[np.ndarray], np.ndarray]:
    """
    Devuelve phi(r) para el kernel especificado
    name puede ser: 'gaussian', 'cubic', 'quintic', 'multiquadric'
    """
    n=name.lower()
    if n=='gaussian':
        def phi(r):
            t = epsilon*r
            return np.exp(-(t*t))
        return phi
    elif n=='cubic':
        def phi(r):
            return r**3
        return phi
    elif n=='quintic':
        def phi(r):
            return r**5
        return phi
    elif n=='multiquadric':
        def phi(r):
            t = epsilon*r
            return np.sqrt(1.0 + t*t)
        return phi
    raise ValueError(f"Kernel '{name}' no reconocido")

# Obtener las distancias entre puntos
def pairwise_dist(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X)
    diff = X[:, None, :] - X[None, :, :]
    return np.sqrt(np.sum(diff*diff, axis=2))

# Diagonalizar sin inversión LL^T
def _diag_inv_from_chol(L: np.ndarray, method: str = "loop") -> np.ndarray:
    """
    diag((L L^T)^{-1}) sin invertir explícitamente.
    method='I': resuelve L Z = I y usa diag = sum(Z^2, axis=0) (rápido, más memoria).
    method='loop': resuelve L z_i = e_i por columnas (menos memoria).
    """
    n = L.shape[0]
    if method == "I":
        Z = solve_triangular(L, np.eye(n), lower=True)
        return np.sum(Z*Z, axis=0)
    elif method == "loop":
        diagM = np.empty(n)
        for i in range(n):
            ei = np.zeros(n); ei[i] = 1.0
            zi = solve_triangular(L, ei, lower=True)
            diagM[i] = np.dot(zi, zi)
        return diagM
    else:
        raise ValueError("method debe ser 'I' o 'loop'")

In [21]:
# --- PRESS / LOO ---
def press_loocv(
    X: np.ndarray, y: np.ndarray, kernel: str = "gaussian",
    epsilon: float = 1.0, lam: float = 0.0,
    diag_method: str = "loop", return_all: bool = False
):
    """
    Calcula PRESS y residuos LOO sin reentrenar.
    Retorna: press, (e_loo, yhat_loo, c, diagM) si return_all=True.
    """
    X = np.asarray(X); y = np.asarray(y).ravel()
    n = len(y)
    phi = kernel_factory(kernel, epsilon)
    r = pairwise_dist(X)
    K = phi(r)
    Klam = K + lam * np.eye(n)
    L = cholesky(Klam, lower=True)
    # c = (K+λI)^{-1} y por sustituciones triangulares
    w = solve_triangular(L, y, lower=True)
    c = solve_triangular(L.T, w, lower=False)
    # diagM = diag((K+λI)^{-1})
    diagM = _diag_inv_from_chol(L, method=diag_method)
    e_loo = c / diagM
    press = float(e_loo @ e_loo)
    if return_all:
        yhat_loo = y - e_loo
        return press, e_loo, yhat_loo, c, diagM
    return press

def grid_search_press(
    X: np.ndarray, y: np.ndarray, kernel: str,
    eps_grid: np.ndarray, lam_grid: np.ndarray,
    diag_method: str = "loop"
) -> Tuple[float, float, float, List[Tuple[float,float,float]]]:
    """
    Devuelve (best_press, best_eps, best_lam, resultados[(press, eps, lam), ...]) ordenados por press.
    """
    results: List[Tuple[float,float,float]] = []
    for eps in eps_grid:
        for lam in lam_grid:
            try:
                pr = press_loocv(X, y, kernel, eps, lam, diag_method=diag_method, return_all=False)
                results.append((pr, float(eps), float(lam)))
            except np.linalg.LinAlgError:
                # No SPD (p.ej. λ demasiado pequeño)
                pass
    if not results:
        raise RuntimeError("Ninguna combinación produjo sistema SPD. Incrementa λ.")
    results.sort(key=lambda t: t[0])
    best_press, best_eps, best_lam = results[0]
    return best_press, best_eps, best_lam, results

In [22]:
# --- Predicción con el modelo elegido ---
def rbf_fit(
    X: np.ndarray, y: np.ndarray, kernel: str, epsilon: float, lam: float
) -> Tuple[np.ndarray, Callable[[np.ndarray], np.ndarray]]:
    """
    Ajusta RBF con (kernel, ε, λ) y devuelve (c, predictor(Xq) -> yq).
    """
    X = np.asarray(X); y = np.asarray(y).ravel()
    n = len(y)
    phi = kernel_factory(kernel, epsilon)
    K = phi(pairwise_dist(X))
    Klam = K + lam * np.eye(n)
    L = cholesky(Klam, lower=True)
    w = solve_triangular(L, y, lower=True)
    c = solve_triangular(L.T, w, lower=False)

    def predict(Xq: np.ndarray) -> np.ndarray:
        Xq = np.asarray(Xq)
        # distancias query-train
        diff = Xq[:, None, :] - X[None, :, :]
        R = np.sqrt(np.sum(diff*diff, axis=2))
        Kqx = phi(R)
        return Kqx @ c

    return c, predict

In [ ]:
# Barrido (gaussiana como ejemplo)
eps_grid = np.logspace(-2, 2, 13)
lam_grid = np.r_[0.0, np.logspace(-12, -3, 10)]

best_press, best_eps, best_lam, _ = grid_search_press(x, mu, "gaussian", eps_grid, lam_grid)
print(best_press, best_eps, best_lam)

# Ajuste final y predicción
c, predict = rbf_fit(x, mu, "gaussian", best_eps, best_lam)
y_fit = predict(x)
rmse_train = np.sqrt(np.mean((y_fit - mu)**2))

# LOO/PRESS detallado con esos hiperparámetros
press, e_loo, yhat_loo, c2, diagM = press_loocv(x, mu, "gaussian", best_eps, best_lam, return_all=True)
rmse_loo = np.sqrt(press / len(mu))
print("RMSE train:", rmse_train, "RMSE LOO:", rmse_loo)